In [1]:
import numpy as np
import ctypes
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import astropy.io.fits as pf

In [2]:
covariance = np.loadtxt("input_data/covariance_cosmic_shear_PMEpaper.dat")
precision  = np.linalg.inv(np.matrix(covariance))
N_data = len(covariance[:,0])
fiducial_dv = np.loadtxt("input_data/DES_shear-shear_a1.0_b0.5_data_vector")[:,1]
derivatives = np.loadtxt("input_data/derivatives.dat")
Omega_m_fid = 0.3156
sigma_8_fid = 0.831
w0_fid      = -1.0

In [3]:
# linearize model for the data vector
def linearized_model(params):
    dOm = params[0]-Omega_m_fid
    ds8 = params[1]-sigma_8_fid
    dw  = params[2]-w0_fid
    return fiducial_dv + derivatives[:,0]*dOm + derivatives[:,1]*ds8 + derivatives[:,2]*dw

# treat this as a black box with which you
# generate your simulated data vectors
def simulator(params):
    expec = linearized_model(params)
    return np.random.multivariate_normal(expec, covariance)